In [1]:
print("testing")

testing


In [2]:
%pwd

'd:\\HealthLine Assistant\\HealthLine-Assistant\\research'

In [3]:
cd ../

d:\HealthLine Assistant\HealthLine-Assistant


In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data from PDF
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf_file(data='Data/')

In [ ]:
#extracted_data

In [7]:
#Split the data into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks: ", len(text_chunks))

Length of Text Chunks:  6973


In [9]:
%pip install --force-reinstall sentence-transformers

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.2.1-cp310-cp310-win_amd64.whl.metadata (9.1 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using 

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.60 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.


In [59]:
import sys
import os
print(f"Python version: {sys.version}")
print(f"Python path: {sys.executable}")
print(f"Current working directory: {os.getcwd()}")

# Verify sentence-transformers installation
try:
    import sentence_transformers
    print(f"sentence-transformers version: {sentence_transformers.__version__}")
    
    # Additional verification
    from sentence_transformers import SentenceTransformer
    print("SentenceTransformer class successfully imported")
except ImportError as e:
    print(f"Import error: {e}")
except Exception as e:
    print(f"Other error: {str(e)}")

Python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
Python path: c:\Users\futur\miniconda3\envs\healthLineAssistant\python.exe
Current working directory: d:\HealthLine Assistant\HealthLine-Assistant
sentence-transformers version: 4.1.0
SentenceTransformer class successfully imported


In [60]:
%pip install langchain-huggingface

In [61]:
from langchain.embeddings import HuggingFaceEmbeddings

In [62]:
def download_huggingface_embeddings():
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-MiniLM-L6-v2',
            cache_folder=None  # Let it use default cache
        )
        return embeddings
    except Exception as e:
        print(f"Error creating embeddings: {str(e)}")
        raise

In [63]:

embeddings = download_huggingface_embeddings()
print("Embeddings object created successfully")

Embeddings object created successfully


In [64]:
query_result = embeddings.embed_query("What is the capital of France?")
print("Length", len(query_result))

Length 384


In [65]:
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [67]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "healthlineassistant"

# Only create the index if it does not already exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Replace with your model dimensions
        metric="cosine",  # Replace with your model metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")

In [68]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [69]:
# Embed each chunk and upsert into Pinecone
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [70]:
# Load existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [71]:
docsearch

In [72]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [73]:
retrieved_docs = retriever.invoke("What is cancer?")

In [74]:
retrieved_docs

[Document(id='cbc56e8e-12b4-4995-9b1d-b7eae7a9afe4', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 20.0, 'page_label': '21', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\Medicine_book.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Cancer, by definition, is a disease of the genes. A\ngene is a small part of DNA, which is the master mole-\ncule of the cell. Genes make “proteins,” which are the\nultimate workhorses of the cells. It is these proteins that\nallow our bodies to carry out all the many processes that\npermit us to breathe, think, move, etc.\nThroughout people’s lives, the cells in their bodies\nare growing, dividing, and replacing themselves. Many\ngenes produce proteins that are involved in controlling'),
 Document(id='f1a9cf9b-d1da-4398-a561-9b239ef56f8e', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'key

In [75]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens=500)

In [76]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrived context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [77]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [78]:
response = rag_chain.invoke({"input": "When is Avengers infinity war release date?"})
print(response["answer"])



System: I don't know.
